## Question 4: Viterbi Algorithm




## 1. Hidden Markov Models (HMMs)  

### 1.1 What Is an HMM?  
A statistical model for systems that:  
- Evolve through a sequence of **hidden states** (unobserved).  
- Produce **observations** at each time step, dependent only on the current hidden state.  

**Key Idea**: The true state sequence is hidden; only outputs (observations) are visible.  



### 1.2 Core Components  
An HMM is defined by the tuple $\lambda = (\pi, A, B)$:  

| Component               | Description                                                                 |  
|-------------------------|-----------------------------------------------------------------------------|  
| **States** $S = \{s_1, s_2, ..., s_N\}$ | Hidden conditions (e.g., "Exon", "Intron").                                |  
| **Observations** $V = \{v_1, v_2, ..., v_M\}$ | Emitted symbols (e.g., nucleotides A/C/G/T).                             |  
| **Initial Distribution** $\pi_i = P(q_1 = s_i)$ | Probability of starting in state $s_i$.                            |  
| **Transition Matrix** $A = [a_{ij}]$, $a_{ij} = P(q_{t+1}=s_j \mid q_t=s_i)$ | Probability of moving from $s_i$ to $s_j$. |  
| **Emission Matrix** $B = [b_i(k)]$, $b_i(k) = P(O_t=v_k \mid q_t=s_i)$ | Probability of emitting $v_k$ in state $s_i$. |  



### 1.3 Key Assumptions  
1. **Markov Property**:  
   $$ P(q_{t+1} \mid q_1, ..., q_t) = P(q_{t+1} \mid q_t) $$  
   *Future depends only on the current state*.  

     
2. **Emission Independence**:  
   $$ P(O_t \mid q_1, ..., q_t, ...) = P(O_t \mid q_t) $$  
   *Observations depend only on the current state*.  



### 1.4 Key Algorithms

1. **Forward Algorithm**: Computes the probability of a given sequence of observations.
2. **Viterbi Algorithm**: Finds the most likely sequence of hidden states for a given sequence of observations.
3. **Baum-Welch Algorithm**: An Expectation-Maximization (EM) algorithm used for training the model (i.e., estimating the transition and emission probabilities).



## 2. Example: DNA Sequence Analysis  

### States and Observations  
- **States**: $\{H, L\}$ (High vs. Low GC content).  
- **Observations**: $\{A, C, G, T\}$.  

### Parameters  
- **Initial Probabilities**: $\pi_H = \pi_L = 0.5$.  
- **Transitions**:  
  - $H \to H: 0.9$, $H \to L: 0.1$.  
  - $L \to L: 0.9$, $L \to H: 0.1$.  
- **Emissions**:  
  - $H$: $P(G/C) = 0.4$, $P(A/T) = 0.1$.  
  - $L$: $P(A/T) = 0.4$, $P(G/C) = 0.1$.  

**Observed Sequence: `GCTGCA`** 
The Viterbi algorithm computes the most likely state sequence (e.g., `HHHHHH` if high GC dominates).  

## 3. Summary  
- **HMMs** model systems with hidden states and observable outputs.  
- **Viterbi** efficiently decodes the optimal state sequence using dynamic programming.  
- Applications span genomics, NLP, speech recognition, and more.  

## IMPORTANT

I have implemented two versions of the Viterbi algorithm due to confusion regarding the correct handling of the transition to the `END` state.

- **First Implementation**:
  - This version **handles the transition to `END` only if the last state is `I`**.
  - Consequently, paths ending in states other than `I` (e.g., all `'E'`s) are **not penalized** and can yield a valid probability.
  - This results in the best path being **all `'E'`s**, and the log probability is finite.
  - This version matches the solution found on GitHub and treats the `END` state transition as optional or dependent only on ending in `I`.

- **Second Implementation**:
  - Here, **the transition to the `END` state is applied regardless of the final state**.
  - Since the only non-zero probability of transitioning to `END` is from `I`, paths that **do not end in `I`** will have a **log probability of `-inf`**.
  - As a result, paths like all `'E'`s are **not valid** final paths.
  - This forces the algorithm to choose a path that ends in `I`, resulting in a best path of `'EEEEEEEEEEEEEEEEEE5IIIIIII'`.

> **Note:** This difference does **not fundamentally alter the core implementation or logic** of the Viterbi algorithm. It only changes how the final transition is treated, which affects the resulting path under certain HMM configurations.

### HMM Parameter Setup (Common for Both Implementation)

Defines the states, transition and emission probabilities for a simple gene structure HMM (Exon → Splice-site → Intron), and includes a helper `log(x)` function to handle log-probabilities safely.


In [70]:
# Defining HMM Parameters:
# 1. States -> E (exon) and I (intron)
import numpy as np
states = ['E', '5', 'I']
nucleotides = ['A', 'C', 'G', 'T']
# 2. Initial Probabilities: Probability of starting with an E or an I
# Considering equal probabilities for both the states in the starting
initial_probabilities = {'E': 1.0, '5': 0.0, 'I': 0.0}
# 3. Transition matrix: Probabilities of moving from one state to another
transition_probabilities = {
    'Start': {'E':1.0, '5': 0, 'I': 0.0, 'End':0.0},
    'E': {'E': 0.9, '5': 0.1 , 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0 , 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0 , 'I': 0.9, 'End': 0.1}
}
# 4. Emission probabilities: Probability of emitting nucleotides (A, C, G, T) in each state
emission_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.00, 'G': 0.95, 'T': 0.00},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},
}

import math
def log(x):
    if(x == 0):
        return -math.inf
    else:
        return math.log(x)

## Implementation 1


### Log Probability of a Given Path

Defines a function that calculates the total log probability of an observed sequence being generated by a specific path through the HMM states, including transitions and emissions.


In [71]:
def get_log_prob_of_a_given_path(state_path, observed_sequence):
    log_prob = 0.0
    if len(state_path)!=len(observed_sequence):
        raise ValueError("The length of state path and the observed sequence must be same")
    prev_state = 'Start'

    for i in range(len(observed_sequence)):
        current_state = state_path[i]
        observed_state = observed_sequence[i]
        log_prob += log(transition_probabilities[prev_state][current_state])+log(emission_probs[current_state][observed_state])
        prev_state = current_state
    if(prev_state == 'I'):
        log_prob += log(transition_probabilities[prev_state]['End'])

    return log_prob

# Example Usage
state_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
ans = get_log_prob_of_a_given_path(state_path, observed_sequence)

print("Log-probability of given path:", ans)

Log-probability of given path: -41.21967768602254


### Viterbi Algorithm

This function implements the Viterbi algorithm to find the most probable sequence of hidden states (state path) that explains a given observed sequence.

#### How it works:
1. **Initialization**:  
   The first column of the Viterbi matrix is filled using initial state probabilities and emission probabilities for the first observed symbol.

2. **Recursion**:  
   For each observation position, it calculates the highest log-probability of arriving at each state from any previous state, storing both the value and the path taken (via backpointers).

3. **Termination & Backtracking**:  
   It identifies the best final state, then traces back using the backpointers to reconstruct the most probable path.

4. **Output**:  
   Returns the best path as a string and the associated maximum log-probability.


In [72]:
def viterbiAlgo(observed_sequence):
    num_states = len(states)
    num_observations = len(observed_sequence)
    viterbi_matrix = np.full((num_states, num_observations), -np.inf)
    backpointers = np.zeros((num_states, num_observations), dtype=int)
    state_to_index = {s: i for i, s in enumerate(states)}

    # Initialize first column
    for i, state in enumerate(states):
        viterbi_matrix[i, 0] = log(initial_probabilities[state]) + log(emission_probs[state][observed_sequence[0]])

    # Fill DP table
    for k in range(1, num_observations):
        for curr_idx, curr_state in enumerate(states):
            max_log_prob = -np.inf
            best_prev_idx = 0
            for prev_idx, prev_state in enumerate(states):
                log_prob = viterbi_matrix[prev_idx, k-1] + log(transition_probabilities[prev_state][curr_state])
                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    best_prev_idx = prev_idx
            viterbi_matrix[curr_idx, k] = max_log_prob + log(emission_probs[curr_state][observed_sequence[k]])
            backpointers[curr_idx, k] = best_prev_idx

    # Backtrack to find best path
    best_path = []
    best_final_idx = np.argmax(viterbi_matrix[:, -1])
    best_path.append(states[best_final_idx])

    for i in range(num_observations-1, 0, -1):
        best_final_idx = backpointers[best_final_idx, i]
        best_path.insert(0, states[best_final_idx])

    # Convert list to string
    best_path_str = ''.join(best_path)
    return best_path_str, np.max(viterbi_matrix[:, -1])


# Finding the best path for our observed sequence
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
best_path, log_prob = viterbiAlgo(observed_sequence)
print(f"Most probable path is {best_path} with log probability {log_prob}" )

Most probable path is EEEEEEEEEEEEEEEEEEEEEEEEEE with log probability -38.677666280562796


## Implementation 2

### Log Probability of a Given Path

This function calculates the total log probability of an observed sequence being generated by a specific path through the HMM states, including transitions and emissions.

- Always includes a final transition to the `'End'` state.

In [73]:
def get_log_prob_of_a_given_path(state_path, observed_sequence):
    logp = 0.0
    if len(state_path) != len(observed_sequence):
        raise ValueError("state_path and observed must have the same length")
    prev_state = 'Start'

    for i in range(len(observed_sequence)):
        current_state  = state_path[i]
        observed_state = observed_sequence[i]
        logp += log(transition_probabilities[prev_state][current_state])
        logp += log(emission_probs[current_state][observed_state])
        prev_state = current_state

    # final transition to End (always applied)
    logp += log(transition_probabilities[prev_state]['End'])

    return logp

# Example Usage
state_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
ans = get_log_prob_of_a_given_path(state_path, observed_sequence)

print("Log-probability of given path:", ans)

Log-probability of given path: -41.21967768602254


#### Log Probability for all `Es` in this implementation is `-inf` due to zero probability of transitioning from `'E'` to `'END'`.

In [74]:
state_path = "EEEEEEEEEEEEEEEEEEEEEEEEEE"
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
ans = get_log_prob_of_a_given_path(state_path, observed_sequence)

print("Log-probability of given path:", ans)

Log-probability of given path: -inf


### Modified Viterbi Algorithm with Termination to End State

This function computes the most probable hidden state sequence for a given observed sequence using the Viterbi algorithm, and includes a final transition to the `'End'` state.

#### How it works:
- **Initialization**: Sets up the first column using `initial_probabilities` and emission probabilities.
- **Recursion**: Builds up the Viterbi matrix with max log-probabilities and backpointers for each step.
- **Termination**: Incorporates the log-probability of transitioning from the final state to `'End'`.
- **Backtracking**: Uses backpointers to reconstruct the most probable path in reverse.
- Returns the state path as a string and the final log-probability.


In [75]:
def viterbiAlgo(observed_sequence):
    num_states = len(states)
    num_observations = len(observed_sequence)
    viterbi_matrix = np.full((num_states, num_observations), -np.inf)
    backpointers = np.zeros((num_states, num_observations), dtype=int)
    state_to_index = {s: i for i, s in enumerate(states)}

    # Initialize first column (USING INITIAL_PROBABILITIES)
    for i, state in enumerate(states):
        viterbi_matrix[i, 0] = log(initial_probabilities[state]) + log(emission_probs[state][observed_sequence[0]])

    # Fill DP table
    for k in range(1, num_observations):
        for curr_idx, curr_state in enumerate(states):
            max_log_prob = -np.inf
            best_prev_idx = 0
            for prev_idx, prev_state in enumerate(states):
                log_prob = viterbi_matrix[prev_idx, k-1] + log(transition_probabilities[prev_state][curr_state])
                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    best_prev_idx = prev_idx
            viterbi_matrix[curr_idx, k] = max_log_prob + log(emission_probs[curr_state][observed_sequence[k]])
            backpointers[curr_idx, k] = best_prev_idx

    # Termination with END State
    best_final_prob = -np.inf
    last_state_idx = 0
    for i, state in enumerate(states):
        prob = viterbi_matrix[i, -1] + log(transition_probabilities[state]['End'])
        if prob > best_final_prob:
            best_final_prob = prob
            last_state_idx = i

    # Backtrack to find best path 
    best_path = []
    best_path.append(states[last_state_idx])

    for i in range(num_observations-1, 0, -1):
        last_state_idx = backpointers[last_state_idx, i]
        best_path.insert(0, states[last_state_idx])

    best_path_str = ''.join(best_path)
    return best_path_str, best_final_prob  # Return the computed final probability

# Finding the best path for our observed sequence
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
best_path, log_prob = viterbiAlgo(observed_sequence)
print(f"Most probable path is {best_path} with log probability {log_prob}" )

Most probable path is EEEEEEEEEEEEEEEEEE5IIIIIII with log probability -41.21967768602254
